In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [3]:
train_filename = "../data/samples/train_transactions_gt2018.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "../data/samples/test_transactions_gt2018.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

feature_description = {
    'context_item_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_item_price': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_item_quantity': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_department_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_discount_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_return_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),   
    'label_item_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

In [4]:
def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    "context_item_id": tf.strings.as_string(x["context_item_id"]),
    "label_item_id": tf.strings.as_string(x["label_item_id"])
})

test_ds = test.map(_parse_function).map(lambda x: {
    "context_item_id": tf.strings.as_string(x["context_item_id"]),
    "label_item_id": tf.strings.as_string(x["label_item_id"])
})

In [5]:
items_filename = "../data/samples/items.tfrecord"
items_tf = tf.data.TFRecordDataset(items_filename)
item_feature_description = {
    'item_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'item_fullprice' : tf.io.FixedLenFeature([1], tf.float32, default_value=0),
    'item_description': tf.io.FixedLenFeature([1], tf.string, default_value='None')}
def item_parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, item_feature_description)

items_ds = items_tf.map(item_parse_function).map(lambda x: {
    "item_id": tf.strings.as_string(x["item_id"]),
    "item_fullprice": x["item_fullprice"],
    "item_description": x["item_description"],
})
items = items_ds.map(lambda x: x["item_id"])
item_ids = items.batch(1_000)
unique_item_ids = np.unique(np.concatenate(list(item_ids)))

In [6]:
embedding_dimension = 32

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=unique_item_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_item_ids) + 1, embedding_dimension), 
    tf.keras.layers.GRU(embedding_dimension),
])

candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_item_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_item_ids) + 1, embedding_dimension)
])

metrics = tfrs.metrics.FactorizedTopK(
  candidates=items_tf.batch(128).map(candidate_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics,
)

In [8]:
class RetrievalModel(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task

    def compute_loss(self, features, training=False):
        item_history = features["context_item_id"]
        next_item_label = features["label_item_id"]

        query_embedding = self._query_model(item_history)       
        candidate_embedding = self._candidate_model(next_item_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

In [42]:
model = RetrievalModel(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [43]:
cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()

In [44]:
model.fit(cached_train, epochs=3)

Epoch 1/3
89/89 [==============================] - 107s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 106952.3023 - regularization_loss: 0.0000e+00 - total_loss: 106952.3023
Epoch 2/3
89/89 [==============================] - 83s 933ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 98517.1934 - regularization_loss: 0.0000e+00 - total_loss: 98517.1934
Epoch 3/3
89/89 [==============================] - 83s 933ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 

In [45]:
model.evaluate(cached_test, return_dict=True)

110/110 [==============================] - 89s 790ms/step - factorized_top_k/top_1_categorical_accuracy: 0.9144 - factorized_top_k/top_5_categorical_accuracy: 0.9144 - factorized_top_k/top_10_categorical_accuracy: 0.9144 - factorized_top_k/top_50_categorical_accuracy: 0.9144 - factorized_top_k/top_100_categorical_accuracy: 0.9144 - loss: 15602.0431 - regularization_loss: 0.0000e+00 - total_loss: 15602.0431


{'factorized_top_k/top_1_categorical_accuracy': 0.9144463539123535,
 'factorized_top_k/top_5_categorical_accuracy': 0.9144463539123535,
 'factorized_top_k/top_10_categorical_accuracy': 0.9144463539123535,
 'factorized_top_k/top_50_categorical_accuracy': 0.9144463539123535,
 'factorized_top_k/top_100_categorical_accuracy': 0.9144463539123535,
 'loss': 16.664541244506836,
 'regularization_loss': 0,
 'total_loss': 16.664541244506836}

In [47]:
for x in items_ds.take(5).as_numpy_iterator():
  pprint.pprint(x)

{'item_description': array([b'disp glue brush 1/2in 48pcs'], dtype=object),
 'item_fullprice': array([12.99], dtype=float32),
 'item_id': array([b'8663'], dtype=object)}
{'item_description': array([b'purpleheart 3/4x3x36'], dtype=object),
 'item_fullprice': array([16.99], dtype=float32),
 'item_id': array([b'101069'], dtype=object)}
{'item_description': array([b'walnut rustic 4/4'], dtype=object),
 'item_fullprice': array([0.], dtype=float32),
 'item_id': array([b'80787'], dtype=object)}
{'item_description': array([b'mahogany 4/4'], dtype=object),
 'item_fullprice': array([0.], dtype=float32),
 'item_id': array([b'74158'], dtype=object)}
{'item_description': array([b'maple hard 8/4'], dtype=object),
 'item_fullprice': array([0.], dtype=float32),
 'item_id': array([b'74180'], dtype=object)}


In [48]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model._query_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((items.batch(100), items.batch(100).map(model._candidate_model)))
)
_ = index(tf.constant(['139716','35287','142953','132041','','','','','','',],shape=(1,10,1)))



In [49]:
foo, titles = index(tf.constant(['139716','35287','142953','132041','','','','','','',],shape=(1,10,1)))
print(foo)
print(titles)

tf.Tensor(
[[3.4319124 3.3678741 3.3579855 3.3417988 3.33075   3.2787607 3.2422504
  3.2389758 3.169146  3.1553702]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[[b'124041']
  [b'144315']
  [b'128556']
  [b'133617']
  [b'106025']
  [b'92246']
  [b'143647']
  [b'129276']
  [b'106227']
  [b'127433']]], shape=(1, 10, 1), dtype=string)


In [50]:
path = '../data/retrieval_model'
tf.saved_model.save(index, path)

INFO:tensorflow:Assets written to: ../data/retrieval_model\assets


INFO:tensorflow:Assets written to: ../data/retrieval_model\assets


In [ ]:
foo, titles = loaded(tf.constant(['139716','35287','142953','132041','','','','','','',],shape=(1,10,1)))

In [ ]:
print (titles)